In [2]:
import pandas as pd

# Load saved data (works even in a fresh kernel)
SPY_data = pd.read_csv("SPY_prices.csv", parse_dates=["Date"]).sort_values("Date")  # parse_dates makes Date datetime [web:74]

df = SPY_data.copy()

# Monthly schedule
schedule = pd.DataFrame({
    "ContribDate": pd.date_range(
        start=df["Date"].min().to_period("M").to_timestamp(),
        end=df["Date"].max(),
        freq="MS"
    )
}).sort_values("ContribDate")

# Align contrib date -> next trading day close (merge_asof requires sorted keys). [web:27]
trades = pd.merge_asof(
    schedule,
    df.rename(columns={"Date": "TradeDate"}).sort_values("TradeDate"),
    left_on="ContribDate",
    right_on="TradeDate",
    direction="forward"
).dropna(subset=["TradeDate", "Close"]).copy()

# DCA mechanics
trades["Contribution"] = 1.0
trades["SharesBought"] = trades["Contribution"] / trades["Close"]
trades["CumShares"] = trades["SharesBought"].cumsum()
trades["CumContrib"] = trades["Contribution"].cumsum()
trades["PortfolioValue"] = trades["CumShares"] * trades["Close"]
trades["Return"] = trades["PortfolioValue"] / trades["CumContrib"] - 1.0

trades.tail(10)



,ContribDate,TradeDate,Close,Contribution,SharesBought,CumShares,CumContrib,PortfolioValue,Return
388,2025-05-01,2025-05-01,558.469971,1.0,0.001791,3.141932,389.0,1754.674465,3.510731
389,2025-06-01,2025-06-02,592.710022,1.0,0.001687,3.143619,390.0,1863.254365,3.777575
390,2025-07-01,2025-07-01,617.650024,1.0,0.001619,3.145238,391.0,1942.656226,3.968430
391,2025-08-01,2025-08-01,621.719971,1.0,0.001608,3.146846,392.0,1956.457175,3.990962
392,2025-09-01,2025-09-02,640.270020,1.0,0.001562,3.148408,393.0,2015.831327,4.129342
393,2025-10-01,2025-10-01,668.450012,1.0,0.001496,3.149904,394.0,2105.553444,4.344044
394,2025-11-01,2025-11-03,683.340027,1.0,0.001463,3.151368,395.0,2153.455563,4.451786
395,2025-12-01,2025-12-01,680.270020,1.0,0.001470,3.152838,396.0,2144.780841,4.416113
396,2026-01-01,2026-01-02,683.169983,1.0,0.001464,3.154301,397.0,2154.923955,4.428020
397,2026-02-01,2026-02-02,695.409973,1.0,0.001438,3.155739,398.0,2194.532572,4.513901


In [ ]:
#Exporter .csv
portfolio_series = trades[["TradeDate", "PortfolioValue", "CumContrib"]].copy()
portfolio_series = portfolio_series.rename(columns={"TradeDate": "Date"})
portfolio_series.to_csv("strategy_dca_monthly_portfolio.csv", index=False)
portfolio_series.head()